# Import and Set

In [1]:
import sys
sys.path.append('../../')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import numpy as np
import os
import pickle
from torchaudio.models import Conformer
import math
from torch import nn, Tensor
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import Normalizer, StandardScaler
from einops.layers.torch import Rearrange, Reduce
from utils import *
from model import VT_CNN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_class = 2
stock_symbol = '5871.TW'
end_date = '2024-12-31'

init = False
batch_size = 8
lr = 0.0001
num_epochs = 50

# Init and Data

In [2]:
# 擷取股票價格資訊
stock_price_data = fetch_stock_price(stock_symbol=stock_symbol, start_date='2012-01-02',end_date=end_date)

stock_price_data['do'] = stock_price_data['Open'].pct_change()
stock_price_data['dh'] = stock_price_data['High'].pct_change()
stock_price_data['dl'] = stock_price_data['Low'].pct_change()
stock_price_data['dc'] = stock_price_data['Close'].pct_change()
stock_price_data['dv'] = stock_price_data['Volume'].pct_change()

stock_price_data['do_1'] = stock_price_data['do'].shift(-1)
stock_price_data['dc_1'] = stock_price_data['dc'].shift(-1)
stock_price_data['(o-c)_1'] = (stock_price_data['Open'].shift(-1) - stock_price_data['Close'])/stock_price_data['Close']

stock_price_data = stock_price_data.dropna()

# df = stock_price_data.iloc[:,7:]
df = stock_price_data
# Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN values
df = df.dropna()
scaler = StandardScaler()
scaler.fit(df[['do', 'dh', 'dl', 'dc', 'dv']][:2000])
df[['do', 'dh', 'dl', 'dc', 'dv']] = scaler.fit_transform(df[['do', 'dh', 'dl', 'dc', 'dv']])

# Scaler

def processing():
    """
    If not use a function, torch.cuda.empty not work
    """
    x, y, date = window_x_y(df, 224)
    X = process_x(x)
    X, x_test, y, y_test = train_test(X, y)
    x_train, x_valid, y_train, y_valid = train_valid(X, y)

    trainloader, validloader, testloader = (
        loader(x_train.cpu(), torch.tensor(y_train).to(dtype=torch.float32), batch_size=batch_size), 
        loader(x_valid.cpu(), torch.tensor(y_valid).to(dtype=torch.float32), batch_size=batch_size),
        loader(x_test.cpu(), torch.tensor(y_test).to(dtype=torch.float32), batch_size=batch_size)
        )    
    test_date = df.index[-len(y_test):]
    return trainloader, validloader, testloader, test_date

trainloader, validloader, testloader, test_date = processing()
torch.cuda.empty_cache()

/home/jacob/anaconda3/envs/mlntu/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/tmp/ipykernel_75809/3398650488.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['do', 'dh', 'dl', 'dc', 'dv']] = scaler.fit_transform(df[['do', 'dh', 'dl', 'dc', 'dv']])
100%|██████████| 2725/2725 [00:22<00:00, 120.34it/s]


# Setting
- Model, Optimizer, ...

In [3]:
# Accelerate
# !pip install accelerate==0.2.0
"""
Don't know why 16 not working
"""
fp16_training = True
if fp16_training:
    print('Accelerating')
    from accelerate import Accelerator
    accelerator = Accelerator()
    device = accelerator.device
    model = VT_CNN()
else:
    model = VT_CNN().to(device)

# Load previous
if os.path.exists(f'Temp//Conformer_{stock_symbol}_LastTrainInfo.pk'):
    if init:
        print("Init model")
        lr = lr
        last_epoch = 0
        min_val_loss = 10000.0
        loss_train = []
        loss_valid = []
    else:
        print('Load from last train epoch')
        with open(f'Temp//Conformer_{stock_symbol}_LastTrainInfo.pk', 'rb') as f:
            last_train_info = pickle.load(f)
        lr = last_train_info['lr']
        last_epoch = last_train_info['epoch']
        min_val_loss = last_train_info['min val loss']
        model.load_state_dict(torch.load(f'Temp//Conformer_{stock_symbol}_checkpoint_LastTrainModel.pt'))
        with open(f'Temp//Conformer_{stock_symbol}_TrainValHistLoss.pk', 'rb') as f:
            loss_train_val = pickle.load(f)
        loss_train = loss_train_val['train']
        loss_valid = loss_train_val['valid']
else:
    print("Init model")
    lr = lr
    last_epoch = 0
    min_val_loss = 10000.0
    loss_train = []
    loss_valid = []
    
print(f'Last train epoch: {last_epoch}  '
        f'Last train lr: {lr}   '
        f'Min val loss: {min_val_loss}')

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=len(trainloader)*5, gamma=0.9)        

if fp16_training:
    print('Accelerate Prepare')    
    model, optimizer, trainloader, validloader, scheduler = \
        accelerator.prepare(model, optimizer, trainloader, validloader, scheduler)

Accelerating
Load from last train epoch
Last train epoch: 3  Last train lr: 0.0001   Min val loss: 0.0005550204547152164
Accelerate Prepare


# Train

In [ ]:
for epoch in range(last_epoch, num_epochs):
    # Training phase
    model.train()
    loss_train_e = 0
    for batch_x, batch_y in tqdm(trainloader):
        optimizer.zero_grad()
        outputs = model(batch_x)

        # Loss
        loss = criterion(outputs, batch_y)
        if fp16_training:
            accelerator.backward(loss)
        else:
            loss.backward()
        optimizer.step()
        scheduler.step()
        loss_train_e += loss.item()
        
    loss_train_e /= len(validloader)
    loss_train.append(loss_train_e)
    
    loss_valid_e = 0
    with torch.no_grad():
        model.eval()
        for batch_x_val, batch_y_val in tqdm(validloader):
            # batch_x_val = mask(batch_x_val)
            outputs_val = model(batch_x_val)
            loss = criterion(outputs_val, batch_y_val)
            loss_valid_e += loss.item()
        loss_valid_e /= len(validloader)
        loss_valid.append(loss_valid_e)
            
        torch.save(model.state_dict(), f'Temp/Conformer_{stock_symbol}_checkpoint_LastTrainModel.pt')
        if loss_valid_e < min_val_loss:
            min_val_loss = loss_valid_e
            print(f'New best model found in epoch {epoch} with val loss: {min_val_loss}')
            torch.save(model.state_dict(), f'ConformerResult/Conformer_{stock_symbol}_best_model.pt')            
        if epoch % 50 == 0:
            pass
            # torch.save(model, f'ConformerResult/Conformerr_{stock_symbol}_checkpoint_{epoch}.pt')
            
    with open(f'Temp/Conformer_{stock_symbol}_TrainValHistLoss.pk', 'wb') as f:
        pickle.dump({'train': loss_train, 'valid': loss_valid}, f)
    with open(f'Temp/Conformer_{stock_symbol}_LastTrainInfo.pk', 'wb') as f:
        pickle.dump({'min val loss': min_val_loss, 'epoch': epoch, 'lr': optimizer.param_groups[0]['lr']}, f)
        
    # Print statistics
    print(f'Epoch [{epoch}/{num_epochs}]',
        f'Training Loss: {loss_train_e:.10f}',
        f'Valid Loss: {loss_valid_e:.10f}')

# Validate Model

In [ ]:
def load_model():
    import torch
    model = torch.load(f'ConformerResult/Conformer_{stock_symbol}_best_model.pt')
    return model
model = load_model()

In [ ]:

import gc
def test():
    dataloader = dataloader_test

    model.eval()
    s_pred = []
    s_true = []
    for x, y in tqdm(dataloader):
        y_pred = model(x)
        s_pred.append(y_pred.detach())
        s_true.append(y)
    y_pred_tensor = torch.concat(s_pred)
    y_test_tensor = torch.concat(s_true)
    accuracy = (torch.sign(y_pred_tensor) == torch.sign(y_test_tensor)).sum() / len(y_test_tensor)
    return y_pred_tensor, accuracy

y_pred, acc = test()
print(acc)

In [ ]:
# Derive y_pred and y_train_pred of shape(N, 2) and numpy type

y_pred_numpy = y_pred.cpu().numpy()

# predict with train set
y_train_pred = model(torch.tensor(X[-100:], dtype = torch.float32))
y_train_numpy = y_train_pred.detach().cpu().numpy()


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Scaling
prediction = pd.DataFrame(y_pred_numpy)
scaler = StandardScaler()
scaler.fit(y_train_numpy)
prediction = pd.DataFrame(scaler.transform(prediction))

# Get the predicted price of O and C and Prediction merge with complete data
prediction.columns = ['pred_do_1', 'pred_dc_1']
prediction['Date'] = date

true_and_pred = pd.merge(df.reset_index(), prediction, on = 'Date', how = 'left')
true_and_pred['pred_o'] = (true_and_pred['Open'] * (1 + true_and_pred['pred_do_1'])).shift(1)
true_and_pred['pred_c'] = (true_and_pred['Close'] * (1 + true_and_pred['pred_dc_1'])).shift(1)
true_and_pred['pred_oc'] = true_and_pred['pred_c'] - true_and_pred['pred_o']
true_and_pred['true_oc'] = true_and_pred['Close'] - true_and_pred['Open']

# Backtest
asset_list = []
df_backtest = true_and_pred[['Open', 'Close', 'true_oc', 'pred_oc']].dropna()
asset = 1
for index, (o, c, true, pred) in df_backtest.iterrows():
    if pred > 0:
        returns = true/o
        asset *= (1 + returns)
    asset_list.append(asset)

print(asset)
plt.plot(asset_list, label = 'resnet')
plt.plot(df_backtest.reset_index()['Close']/df_backtest['Close'].iloc[0], label = 'buy hold')
plt.legend()
plt.savefig('/ConformerResult/test_backtest.jpg')
# plt.show()